In [1]:
from partial import *

/home/mehran/TN_QML/lib/python3.12/site-packages/quimb/core.py:45: UserWarning: 'NUMBA_NUM_THREADS' has been set elsewhere and doesn't match the value 'quimb' has tried to set - 1 vs 4.
  warnings.warn(


In [ ]:
L = 51
bond = 150
p = partial(L = L, bond = bond)

In [3]:
train, test = p.generate_dataset()

Starting dataset generation — this step includes both the training and test sets. Please be patient...


Generating train set for random_points:   0%|          | 0/55 [00:00<?, ?it/s]

Generating train set for random_points: 100%|██████████| 55/55 [00:03<00:00, 17.02it/s]


Train_set is generated in 3.31 seconds.


Generating test set: 100%|██████████| 400/400 [00:20<00:00, 19.25it/s]


Test_set is generated in 21.68 seconds.


In [ ]:
keep = [0] # keep the part that we wanna do partial trace over that
partial_density_matrix_train, partial_density_matrix_test = p.partial_density_matrix(keep=keep)
kernel_partial_train = p.gram_train_partial(partial_train=partial_density_matrix_train, keep=keep)
kernel_partial_test = p.gram_test_partial(partial_train=partial_density_matrix_train,partial_test=partial_density_matrix_test,keep=keep)

In [ ]:
keep = [0,1] # keep the part that we wanna do partial trace over that
partial_density_matrix_train, partial_density_matrix_test = p.partial_density_matrix(keep=keep)
kernel_partial_train = p.gram_train_partial(partial_train=partial_density_matrix_train, keep=keep)
kernel_partial_test = p.gram_test_partial(partial_train=partial_density_matrix_train,partial_test=partial_density_matrix_test,keep=keep)

In [ ]:
keep = [0,1,2] # keep the part that we wanna do partial trace over that
partial_density_matrix_train, partial_density_matrix_test = p.partial_density_matrix(keep=keep)
kernel_partial_train = p.gram_train_partial(partial_train=partial_density_matrix_train, keep=keep)
kernel_partial_test = p.gram_test_partial(partial_train=partial_density_matrix_train,partial_test=partial_density_matrix_test,keep=keep)

In [ ]:
keep = [0,1,2,3] # keep the part that we wanna do partial trace over that
partial_density_matrix_train, partial_density_matrix_test = p.partial_density_matrix(keep=keep)
kernel_partial_train = p.gram_train_partial(partial_train=partial_density_matrix_train, keep=keep)
kernel_partial_test = p.gram_test_partial(partial_train=partial_density_matrix_train,partial_test=partial_density_matrix_test,keep=keep)

In [4]:
keep = [0,1,2,3,4] # keep the part that we wanna do partial trace over that
partial_density_matrix_train, partial_density_matrix_test = p.partial_density_matrix(keep=keep)
kernel_partial_train = p.gram_train_partial(partial_train=partial_density_matrix_train, keep=keep)
kernel_partial_test = p.gram_test_partial(partial_train=partial_density_matrix_train,partial_test=partial_density_matrix_test,keep=keep)

Computing partial density matrices started .....
Tracing over training set (220 items)...


Tracing train set: 100%|██████████| 220/220 [00:00<00:00, 1116.58it/s]


Tracing over test set (400 items)...


Tracing test set: 100%|██████████| 400/400 [00:00<00:00, 792.86it/s]


Partial density matrices computed in 1.63 seconds.
Computing Gram matrix for training set...


Gram Partial Train: 100%|██████████| 48400/48400 [00:00<00:00, 64216.88it/s]


Gram matrix for training set computed in 0.77 seconds.
Computing Gram matrix for testing set...


Gram Partial Test: 100%|██████████| 400/400 [00:02<00:00, 176.39it/s]

Gram matrix for testing set computed in 2.28 seconds.


In [ ]:
keep = [25] # keep the part that we wanna do partial trace over that
partial_density_matrix_train, partial_density_matrix_test = p.partial_density_matrix(keep=keep)
kernel_partial_train = p.gram_train_partial(partial_train=partial_density_matrix_train, keep=keep)
kernel_partial_test = p.gram_test_partial(partial_train=partial_density_matrix_train,partial_test=partial_density_matrix_test,keep=keep)

In [ ]:
keep = [24,25] # keep the part that we wanna do partial trace over that
partial_density_matrix_train, partial_density_matrix_test = p.partial_density_matrix(keep=keep)
kernel_partial_train = p.gram_train_partial(partial_train=partial_density_matrix_train, keep=keep)
kernel_partial_test = p.gram_test_partial(partial_train=partial_density_matrix_train,partial_test=partial_density_matrix_test,keep=keep)

In [ ]:
keep = [24,25,26] # keep the part that we wanna do partial trace over that
partial_density_matrix_train, partial_density_matrix_test = p.partial_density_matrix(keep=keep)
kernel_partial_train = p.gram_train_partial(partial_train=partial_density_matrix_train, keep=keep)
kernel_partial_test = p.gram_test_partial(partial_train=partial_density_matrix_train,partial_test=partial_density_matrix_test,keep=keep)

In [ ]:
keep = [24,25,26,27] # keep the part that we wanna do partial trace over that
partial_density_matrix_train, partial_density_matrix_test = p.partial_density_matrix(keep=keep)
kernel_partial_train = p.gram_train_partial(partial_train=partial_density_matrix_train, keep=keep)
kernel_partial_test = p.gram_test_partial(partial_train=partial_density_matrix_train,partial_test=partial_density_matrix_test,keep=keep)

In [ ]:
keep = [23,24,25,26,27] # keep the part that we wanna do partial trace over that
partial_density_matrix_train, partial_density_matrix_test = p.partial_density_matrix(keep=keep)
kernel_partial_train = p.gram_train_partial(partial_train=partial_density_matrix_train, keep=keep)
kernel_partial_test = p.gram_test_partial(partial_train=partial_density_matrix_train,partial_test=partial_density_matrix_test,keep=keep)